In [35]:
# @title ##**Install mediapipe** { display-mode: "form" }
from IPython.display import clear_output
!pip install mediapipe
clear_output()

In [36]:
# @title ##**Webcam module** { display-mode: "form" }
# import dependencies
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from base64 import b64decode, b64encode
import cv2
import numpy as np
import PIL
import io
import html
import time
 
# function to convert the JavaScript object into an OpenCV image
def js_to_image(js_reply):
  """
  Params:
          js_reply: JavaScript object containing image from webcam
  Returns:
          img: OpenCV BGR image
  """
  # decode base64 image
  image_bytes = b64decode(js_reply.split(',')[1])
  # convert bytes to numpy array
  jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
  # decode numpy array into OpenCV BGR image
  img = cv2.imdecode(jpg_as_np, flags=1)
 
  return img
 
# function to convert OpenCV Rectangle bounding box image into base64 byte string to be overlayed on video stream
def bbox_to_bytes(bbox_array):
  """
  Params:
          bbox_array: Numpy array (pixels) containing rectangle to overlay on video stream.
  Returns:
        bytes: Base64 image byte string
  """
  # convert array into PIL image
  bbox_PIL = PIL.Image.fromarray(bbox_array, 'RGBA')
  iobuf = io.BytesIO()
  # format bbox into png for return
  bbox_PIL.save(iobuf, format='png')
  # format return string
  bbox_bytes = 'data:image/png;base64,{}'.format((str(b64encode(iobuf.getvalue()), 'utf-8')))
 
  return bbox_bytes
 
  
# JavaScript to properly create our live video stream using our webcam as input
def video_stream():
  js = Javascript('''
    var video;
    var div = null;
    var stream;
    var captureCanvas;
    var imgElement;
    var labelElement;
    
    var pendingResolve = null;
    var shutdown = false;
    
    function removeDom() {
       stream.getVideoTracks()[0].stop();
       video.remove();
       div.remove();
       video = null;
       div = null;
       stream = null;
       imgElement = null;
       captureCanvas = null;
       labelElement = null;
    }
    
    function onAnimationFrame() {
      if (!shutdown) {
        window.requestAnimationFrame(onAnimationFrame);
      }
      if (pendingResolve) {
        var result = "";
        if (!shutdown) {
          captureCanvas.getContext('2d').drawImage(video, 0, 0, 640, 480);
          result = captureCanvas.toDataURL('image/jpeg', 0.8)
        }
        var lp = pendingResolve;
        pendingResolve = null;
        lp(result);
      }
    }
    
    async function createDom() {
      if (div !== null) {
        return stream;
      }
 
      div = document.createElement('div');
      div.style.border = '2px solid black';
      div.style.padding = '3px';
      div.style.width = '100%';
      div.style.maxWidth = '600px';
      document.body.appendChild(div);
      
      const modelOut = document.createElement('div');
      modelOut.innerHTML = "<span>Status:</span>";
      labelElement = document.createElement('span');
      labelElement.innerText = 'No data';
      labelElement.style.fontWeight = 'bold';
      modelOut.appendChild(labelElement);
      div.appendChild(modelOut);
           
      video = document.createElement('video');
      video.style.display = 'block';
      video.width = div.clientWidth - 6;
      video.setAttribute('playsinline', '');
      video.onclick = () => { shutdown = true; };
      stream = await navigator.mediaDevices.getUserMedia(
          {video: { facingMode: "environment"}});
      div.appendChild(video);
 
      imgElement = document.createElement('img');
      imgElement.style.position = 'absolute';
      imgElement.style.zIndex = 1;
      imgElement.onclick = () => { shutdown = true; };
      div.appendChild(imgElement);
      
      const instruction = document.createElement('div');
      instruction.innerHTML = 
          '<span style="color: red; font-weight: bold;">' +
          'When finished, click here or on the video to stop this demo</span>';
      div.appendChild(instruction);
      instruction.onclick = () => { shutdown = true; };
      
      video.srcObject = stream;
      await video.play();
 
      captureCanvas = document.createElement('canvas');
      captureCanvas.width = 640; //video.videoWidth;
      captureCanvas.height = 480; //video.videoHeight;
      window.requestAnimationFrame(onAnimationFrame);
      
      return stream;
    }
    async function stream_frame(label, imgData) {
      if (shutdown) {
        removeDom();
        shutdown = false;
        return '';
      }
 
      var preCreate = Date.now();
      stream = await createDom();
      
      var preShow = Date.now();
      if (label != "") {
        labelElement.innerHTML = label;
      }
            
      if (imgData != "") {
        var videoRect = video.getClientRects()[0];
        imgElement.style.top = videoRect.top + "px";
        imgElement.style.left = videoRect.left + "px";
        imgElement.style.width = videoRect.width + "px";
        imgElement.style.height = videoRect.height + "px";
        imgElement.src = imgData;
      }
      
      var preCapture = Date.now();
      var result = await new Promise(function(resolve, reject) {
        pendingResolve = resolve;
      });
      shutdown = false;
      
      return {'create': preShow - preCreate, 
              'show': preCapture - preShow, 
              'capture': Date.now() - preCapture,
              'img': result};
    }
    ''')
 
  display(js)
  
def video_frame(label, bbox):
  data = eval_js('stream_frame("{}", "{}")'.format(label, bbox))
  return data

In [80]:
# @title ##**Hand detection module** { display-mode: "form" }
import cv2
import mediapipe as mp 
import time
import numpy as np
class HandDetector():
  def __init__(self, static_image_mode=False, max_num_hands=2, min_detection_confidence=0.5, min_tracking_confidence=0.5):
    self.static_image_mode_option=static_image_mode
    self.max_number_of_hands= max_num_hands
    self.min_detection_con=min_detection_confidence
    self.min_tracing_con=min_tracking_confidence
    self.mp_hands = mp.solutions.hands
    self.hands=self.mp_hands.Hands(self.static_image_mode_option,self.max_number_of_hands,self.min_detection_con,self.min_tracing_con)

  def findHands(self, img , draw=True):
    image=img
    imgRGB=cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    self.results =self.hands.process(imgRGB)
    if self.results.multi_hand_landmarks:
      ih,iw, c = image.shape
      img_blank=np.zeros((ih,iw,4),np.uint8)
      for hand_landmarks in self.results.multi_hand_landmarks:
        dict_line={}
        for id, lm in enumerate(hand_landmarks.landmark):
                    x_y_list=[]
                    h, w, c = image.shape
                    cx, cy = int(lm.x * w), int(lm.y * h)
                    x_y_list.append(cx)
                    x_y_list.append(cy)
                    dict_line[id]=x_y_list
                    # print(id, cx, cy)
        if draw:
          img_blank=self.Drawing(img_blank,dict_line)         
    return img_blank,dict_line
        
  def Drawing(self,img_blank,dict_line): 
    Hand_Connection_Color= "255,255,255" #@param {type:"string"}
    Hand_Connection_Thickness = 3  # @param {type: "slider", min: 0, max: 5}
    thickness=int(Hand_Connection_Thickness)
    Hand_Connection_Color_list=Hand_Connection_Color.split(",") 
    r=int(Hand_Connection_Color_list[0])
    g=int(Hand_Connection_Color_list[1])
    b=int(Hand_Connection_Color_list[2])
    cv2.line(img_blank,(dict_line[0][0],dict_line[0][1]),(dict_line[1][0],dict_line[1][1]),(r,g,b ),thickness)
    cv2.line(img_blank,(dict_line[0][0],dict_line[0][1]),(dict_line[5][0],dict_line[5][1]),(r,g,b ),thickness)
    cv2.line(img_blank,(dict_line[0][0],dict_line[0][1]),(dict_line[17][0],dict_line[17][1]),(r,g,b ),thickness)
    cv2.line(img_blank,(dict_line[1][0],dict_line[1][1]),(dict_line[2][0],dict_line[2][1]),(r,g,b ),thickness)
    cv2.line(img_blank,(dict_line[2][0],dict_line[2][1]),(dict_line[3][0],dict_line[3][1]),(r,g,b ),thickness)
    cv2.line(img_blank,(dict_line[3][0],dict_line[3][1]),(dict_line[4][0],dict_line[4][1]),(r,g,b ),thickness)
    cv2.line(img_blank,(dict_line[5][0],dict_line[5][1]),(dict_line[6][0],dict_line[6][1]),(r,g,b ),thickness)
    cv2.line(img_blank,(dict_line[6][0],dict_line[6][1]),(dict_line[7][0],dict_line[7][1]),(r,g,b ),thickness)
    cv2.line(img_blank,(dict_line[7][0],dict_line[7][1]),(dict_line[8][0],dict_line[8][1]),(r,g,b ),thickness)
    cv2.line(img_blank,(dict_line[5][0],dict_line[5][1]),(dict_line[9][0],dict_line[9][1]),(r,g,b ),thickness)
    cv2.line(img_blank,(dict_line[9][0],dict_line[9][1]),(dict_line[10][0],dict_line[10][1]),(r,g,b ),thickness)
    cv2.line(img_blank,(dict_line[10][0],dict_line[10][1]),(dict_line[11][0],dict_line[11][1]),(r,g,b ),thickness)
    cv2.line(img_blank,(dict_line[11][0],dict_line[11][1]),(dict_line[12][0],dict_line[12][1]),(r,g,b ),thickness)
    cv2.line(img_blank,(dict_line[9][0],dict_line[9][1]),(dict_line[13][0],dict_line[13][1]),(r,g,b ),thickness)
    cv2.line(img_blank,(dict_line[13][0],dict_line[13][1]),(dict_line[14][0],dict_line[14][1]),(r,g,b ),thickness)
    cv2.line(img_blank,(dict_line[14][0],dict_line[14][1]),(dict_line[15][0],dict_line[15][1]),(r,g,b ),thickness)
    cv2.line(img_blank,(dict_line[15][0],dict_line[15][1]),(dict_line[16][0],dict_line[16][1]),(r,g,b ),thickness)
    cv2.line(img_blank,(dict_line[13][0],dict_line[13][1]),(dict_line[17][0],dict_line[17][1]),(r,g,b ),thickness)
    cv2.line(img_blank,(dict_line[17][0],dict_line[17][1]),(dict_line[18][0],dict_line[18][1]),(r,g,b ),thickness)
    cv2.line(img_blank,(dict_line[18][0],dict_line[18][1]),(dict_line[19][0],dict_line[19][1]),(r,g,b ),thickness)
    cv2.line(img_blank,(dict_line[19][0],dict_line[19][1]),(dict_line[20][0],dict_line[20][1]),(r,g,b ),thickness)
    for i in dict_line:
      x_value=dict_line[i][0]
      y_value=dict_line[i][1]
      #add circle  
      Hand_knuckle_Color= "16,218,232" #@param {type:"string"}
      hand_knuckle_color_list=Hand_knuckle_Color.split(",")
      Hand_knuckle_Radius = 5  # @param {type: "slider", min: 0, max: 15}
      radius=int(Hand_knuckle_Radius)
      cv2.circle(img_blank, (x_value,y_value), radius, (int(hand_knuckle_color_list[0]),int(hand_knuckle_color_list[1]),int(hand_knuckle_color_list[2])), cv2.FILLED)
      #add hand-knuckle coordinates     
      Display_Hand_Knuckle_Coordinates = "Yes"  #@param ['Yes', 'No']
      if  Display_Hand_Knuckle_Coordinates == "Yes":
        cv2.putText(img_blank,str(i),(x_value,y_value-20),cv2.FONT_HERSHEY_PLAIN,1,(0, 255, 0), 1)
      if Display_Hand_Knuckle_Coordinates == "No":
        pass
    return img_blank





# @title ##**Realtime hand detection** { display-mode: "form" }
 
def main():
    video_stream()
    label_html = 'Capturing...'
    bbox = ''
    hand_detector= HandDetector()
    while True:
      js_reply = video_frame(label_html, bbox)
      if not js_reply:
          break
      img = js_to_image(js_reply["img"])
      try:
        img_blank,dict_points= hand_detector.findHands(img , draw=True)
        img_blank[:,:,3] = (img_blank.max(axis = 2) > 0 ).astype(int) * 255
        bbox_bytes = bbox_to_bytes(img_blank)
        bbox = bbox_bytes
      except:
        ih, iw, ic =img.shape
        img_blank=np.zeros((ih,iw,4),np.uint8)
        img_blank[:,:,3] = (img_blank.max(axis = 2) > 0 ).astype(int) * 255
        bbox_bytes = bbox_to_bytes(img_blank)
        bbox = bbox_bytes
 
if __name__ == "__main__":
    main()

<IPython.core.display.Javascript object>